# The Battle of Neighborhood 

## Business problem: Open a restaurent in London

### Methedology: Analyse the total number of hotels in London, then which area is more trending

In [11]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

SyntaxError: invalid syntax (<ipython-input-11-660b90cc212e>, line 5)

In [4]:
CLIENT_ID = 'BRZKRMDSV1E5SRE21E2RSJEIHF1GNYKS0SEK2NGKDKELNILY' # your Foursquare ID
CLIENT_SECRET = 'PTFDKUT453XNBYHGHLUHMHREMP2TYSZ0H001RHAPH4UA55XL' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BRZKRMDSV1E5SRE21E2RSJEIHF1GNYKS0SEK2NGKDKELNILY
CLIENT_SECRET:PTFDKUT453XNBYHGHLUHMHREMP2TYSZ0H001RHAPH4UA55XL


In [6]:
address = 'London, UK'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

NameError: name 'Nominatim' is not defined

In [7]:
latitude = 51.5073219
longitude = -0.1276474
print(latitude, longitude)

36.1699 115.1398


In [ ]:

search_query = 'Hotels'
radius = 500
print(search_query + ' .... OK!')

In [ ]:

# #### Define the corresponding URL

# In[31]:


url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url


In [ ]:
# #### Send the GET Request and examine the results

# In[32]:


results = requests.get(url).json()
results


In [ ]:
# #### Get relevant part of JSON and transform it into a *pandas* dataframe

# In[33]:


# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()


# #### Define information of interest and filter dataframe

# In[34]:



In [ ]:

# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

In [ ]:

# #### Let's visualize the Italian restaurants that are nearby

# In[35]:


dataframe_filtered.name


# In[36]:


venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Conrad Hotel
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map


In [12]:

venue_id = '4fa862b3e4b0ebff2f749f06' # ID of Harry's Italian Pizza Bar
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url


# #### Send GET request for result

# In[38]:


result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']


# ### B. Get the venue's overall rating

# In[39]:


try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')


# That is not a very good rating. Let's check the rating of the second closest Italian restaurant.

# In[40]:


venue_id = '4f3232e219836c91c7bfde94' # ID of Conca Cucina Italian Restaurant
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')


# Since this restaurant has no ratings, let's check the third restaurant.

# In[41]:


venue_id = '3fd66200f964a520f4e41ee3' # ID of Ecco
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')


# Since this restaurant has a slightly better rating, let's explore it further.

# ### C. Get the number of tips

# In[42]:


result['response']['venue']['tips']['count']


# ### D. Get the venue's tips
# > `https://api.foursquare.com/v2/venues/`**VENUE_ID**`/tips?client_id=`**CLIENT_ID**`&client_secret=`**CLIENT_SECRET**`&v=`**VERSION**`&limit=`**LIMIT**

# #### Create URL and send GET request. Make sure to set limit to get all tips

# In[43]:


## Ecco Tips
limit = 15 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()
results


# #### Get tips and list of associated features

# In[44]:


tips = results['response']['tips']['items']

tip = results['response']['tips']['items'][0]
tip.keys()


# #### Format column width and display all tips

# In[45]:


pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered

Hotels .... OK!


NameError: name 'json_normalize' is not defined